In [ ]:
import urllib3
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
url=pd.read_excel("Input.xlsx")

In [ ]:
stopwords_files = ["StopWords_Auditor.txt","StopWords_Currencies.txt","StopWords_DatesandNumbers.txt","StopWords_Generic.txt","StopWords_GenericLong.txt","StopWords_Geographic.txt","StopWords_Names.txt"]
custom_stopwords = set()
for filename in stopwords_files:
    with open(filename,'r',encoding='latin-1') as file:
        custom_stopwords.update(file.read().splitlines())

In [ ]:
results_list = []
for _, row in url.iterrows():
    url_id = row["URL_ID"]
    current_url = row["URL"]
    http = urllib3.PoolManager()
    response = http.request("GET",current_url)
    if response.status == 200:
      html_content = response.data.decode('latin-1')
      soup = BeautifulSoup(html_content, 'html.parser')
      paragraph_tags = soup.find_all(['p','h1'])
      paragraph_text = [tag.get_text() for tag in paragraph_tags]
      paragraph_text = paragraph_text[16:-3]
      paragraph_text = '\n'.join(paragraph_text)
      sentences = sent_tokenize(paragraph_text)
      paragraph_text = paragraph_text.replace('?', '').replace('!', '').replace(',', '').replace('.', '')
      #Positive and Negative words
      words = word_tokenize(paragraph_text)
      #Saving filtered words with url_id as name
      filtered_words = [word for word in words if word.lower() not in custom_stopwords]
      file = open(f"{url_id}.txt", "w", encoding='utf-8')
      file.write(paragraph_text)
      file.close()
      positive_words = set()
      negative_words = set()
      with open('positive-words.txt', 'r', encoding='latin-1') as positive_file:
          positive_words.update(positive_file.read().splitlines())
      with open('negative-words.txt', 'r', encoding='latin-1') as negative_file:
          negative_words.update(negative_file.read().splitlines())
      positive_count = 0
      negative_count = 0
      for word in filtered_words:
          if word.lower() in positive_words:
              positive_count += 1
          elif word.lower() in negative_words:
              negative_count += 1
      #Polarity Score
      polarity_score=(positive_count - negative_count)/ ((positive_count + negative_count) + 0.000001)
      #Word count
      total_word_count=len(filtered_words)
      #Subjective score
      subjective_score=(positive_count + negative_count)/ ((len(filtered_words)) + 0.000001)
      #Average Sentence Length
      sentence_length=len(sentences)
      avg_sentence_length=len(words)/sentence_length
      #Syllable count and Complex word count
      syllable_count=0
      complex_words=0
      for word in filtered_words:
        count=0
        for i in range(len(word)):
          if(word[i]=='a' or word[i]=='e' or word[i] =='i' or word[i] == 'o' or word[i] == 'u'):
                count+=1
          if(i==len(word)-2 and (word[i]=='e' and word[i+1]=='d')):
            count-=1;
          if(i==len(word)-2 and (word[i]=='e' and word[i+1]=='s')):
            count-=1;
        syllable_count+=count
        if(count>2):
            complex_words+=1
      #Syllable per word
      syllable_per_word=syllable_count/total_word_count
      #Percentage of complex words
      percent_complex_words=complex_words/total_word_count
      #Fog index
      fog_index=0.4 * (avg_sentence_length + percent_complex_words)
      #Average number of words
      avg_number_words=len(words)/len(sentences)
      #Average number of words per sentence
      avg_number_words_sentence=len(words)/len(sentences)
      #Personal Pronoun
      personal_pronoun_count=0
      personal_pronouns = ['I','we','We','my','My','ours','Ours','us' ]
      for word in words:
        if word in personal_pronouns:
          personal_pronoun_count+=1
      #Average word length
      total_word_length = sum(len(word) for word in words)
      avg_word_length = total_word_length / len(words)
      result_dict = {
            'URL_ID': url_id,
            'URL': current_url,
            'POSITIVE SCORE': positive_count,
            'NEGATIVE SCORE': negative_count,
            'POLARITY SCORE': polarity_score,
            'SUBJECTIVITY SCORE': subjective_score,
            'AVG SENTENCE LENGTH': avg_sentence_length,
            'PERCENTAGE OF COMPLEX WORDS': percent_complex_words,
            'FOG INDEX': fog_index,
            'AVG NUMBER OF WORDS PER SENTENCE':avg_number_words_sentence,
            'COMPLEX WORD COUNT': complex_words,
            'WORD COUNT': total_word_count,
            'SYLLABLE PER WORD':syllable_per_word,
            'PERSONAL PRONOUNS': personal_pronoun_count,
            'AVG WORD LENGTH': avg_word_length

        }
      results_list.append(result_dict)
    else:
        print(f"Failed to fetch data from {current_url}. Status code: {response.status}")

Failed to fetch data from https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/. Status code: 404
Failed to fetch data from https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/. Status code: 404


In [ ]:
results_df = pd.DataFrame(results_list)
results_df.to_excel("Output Data Structure.xlsx", index=False)